In [ ]:
from collections import OrderedDict
import pytz
import datetime 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipline.api import symbol as zipline_symbol
from zipline.api import order, record, fetch_csv, order_percent, order_target_percent, set_benchmark
from zipline.algorithm import TradingAlgorithm
from zipline.utils.factory import create_simulation_parameters
from zipline.finance import commission
import pyfolio as pf

from query import *

Start = '2008-03-11'
End = '2020-01-10'

symbols=['cotton_settle', 'cattle_settle', 'canola_settle',
       'wheat_settle', 'copper_settle', 'corn_settle', 'oats_settle',
       'oil_settle']

symbols=['rici_settle'
         ,'canola_settle'
         ,'oats_settle'
#          ,'cotton_settle'
         ,'wheat_settle'
         ,'corn_settle'
         ,'copper_settle'
#          ,'Sentiment_settle'
         ,'oil_settle']
#          ,'cattle_settle'
#          ,'rica_settle']

# file to store with signals
signalsPath = getTempFile(outputPath)

# 'date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor'
# columns = 'open,high,low,close,volume,adjClose'.split(",")
path_to_EOD = "/Users/frizzell/forecast_models/data/raw_data_1monthFuture2020-01-10.csv"
df = pd.read_csv(path_to_EOD)
df.index =  df.Date.apply(datetime.datetime.strptime,args=('%Y-%m-%d',))
del df["Date"]
df = df[Start:End]
# panelPrices = getPrices("dumy", symbols, panelStart, panelEnd, columns)
# panelPrices.minor_axis = columns
dateToStringF = lambda x: x.strftime('%Y-%m-%d')

In [ ]:
# df.tail(10)#09
# print(signalsPath)

In [ ]:
# df.tail(10)#07

In [ ]:
#Define some functions
# def zScore(series,span): 
#     ts = ( series - series.ewm(span).mean() ) / series.ewm(span).std()
#     return ( ts - ts.mean() ) / ts.std(ddof=0)
# def returns(series,future_span):
#     return -1.0 * (series - series.rolling(future_span).mean().shift(-1*future_span)) / series.rolling(future_span).std()

# 1monthfutures_20100101_20170101_20180101_forward-2_back-30_lmbda-1e-09_t_shape-uniform_f_shape-uniform_lstm-1_batch_size50

In [ ]:
train_start = pd.to_datetime('2008-01-01', format='%Y-%m-%d', errors='ignore')
train_end = pd.to_datetime('2017-01-01', format='%Y-%m-%d', errors='ignore')
test_start = pd.to_datetime('2018-01-01', format='%Y-%m-%d', errors='ignore')
test_end = pd.to_datetime('2020-01-10', format='%Y-%m-%d', errors='ignore')
# test_end = pd.to_datetime('2019-06-30', format='%Y-%m-%d', errors='ignore')


# 1monthfuturesforward-15_back-15_lmbda-1e-09_t_shape-uniform_f_shape-uniform_batch_size10
# look_forward=300
# look_back=700

look_forward=2
look_back=30

def create_features_old(df,train_start_date,test_start_date,test_end_date,back=30,forward=30):
    X = df.drop(columns=[col for col in df.columns])
    no_repeat = []
    for col_1 in df:
        for col_2 in df:
            if col_1 == col_2:
                continue
            no_repeat.append(col_1)
            if col_2 in no_repeat:
                continue
            X[col_1.split('_')[0]+'-'+col_2.split('_')[0]] = df[col_1]-df[col_2]
#     X = X-X.rolling(back).mean()
#     X = X-X.mean()
    production_date = X[:test_end_date].index[-1*forward] 
            
    return X[train_start_date:test_start_date], \
           X[test_start_date:production_date], \
           X[production_date:]

def create_features(df,train_start_date,train_end_date,test_start_date,test_end_date,back=30,forward=30):
    X = df.drop(columns=[col for col in df.columns])
    no_repeat = []
    for col_1 in df:
        for col_2 in df:
            if col_1 == col_2:
                continue
            no_repeat.append(col_1)
            if col_2 in no_repeat:
                continue
            X[col_1.split('_')[0]+'-'+col_2.split('_')[0]] = df[col_1]-df[col_2]
#     X = X-X.rolling(back).mean()
#     X = X-X.mean()
    production_date = X[:test_end_date].index[-1*forward] 
            
    return X[train_start_date:train_end_date], \
           X[test_start_date:production_date], \
           X[production_date:]

def make_targets_old(df,train_start_date,test_start_date,test_end_date,forward=30):
    Y = -1.0*(df.wheat_settle - df.wheat_settle.rolling(forward).mean().shift(-1*forward))
    
    production_date = Y[:test_end_date].index[-1*forward] 
    
    return Y[train_start_date:test_start_date],Y[train_start_date:test_start_date].index, \
           Y[test_start_date:production_date],Y[test_start_date:production_date].index

def make_targets(df,train_start_date,train_end_date,test_start_date,test_end_date,forward=30):
    
    Y = -1.0*(df.wheat_settle - df.wheat_settle.rolling(forward).mean().shift(-1*forward))
    
    production_date = Y[:test_end_date].index[-1*forward] 
    
    return Y[train_start_date:train_end_date],Y[train_start_date:train_end_date].index, \
           Y[test_start_date:production_date],Y[test_start_date:production_date].index


Y_train,Y_train_dates,Y_test,Y_test_dates = make_targets(df,
                                                  train_start,
                                                  train_end,
                                                  test_start,
                                                  test_end,
                                                  forward=look_forward)

X_train, X_test, X_production = create_features(df,
                                                train_start,
                                                train_end,
                                                test_start,
                                                test_end,
                                                back=look_back,
                                                forward=look_forward)

In [ ]:
from sklearn.preprocessing import QuantileTransformer

feature_transformer = QuantileTransformer(n_quantiles=1000,
                                        output_distribution='uniform',
                                        copy=True)

target_transformer = QuantileTransformer(n_quantiles=1000,
                                        output_distribution='uniform',
                                        copy=True)

In [ ]:
# print(X_train.shape)
# print(X_test.shape)
# # DF = feature_transformer.fit_transform(DF)X
# X_train = feature_transformer.fit_transform(X_train)
# X_test = feature_transformer.transform(X_test)
# print(X_train.shape)
# print(X_test.shape)

print(Y_train.shape)
print(Y_test.shape)
print(X_train.shape)
print(X_test.shape)
# DF = feature_transformer.fit_transform(DF)X
X_train = feature_transformer.fit_transform(X_train)
Y_train = target_transformer.fit_transform(Y_train.values.reshape(-1,1))
X_test = feature_transformer.transform(X_test)
Y_test = target_transformer.transform(Y_test.values.reshape(-1,1))
print(Y_train.shape)
print(X_train.shape)
print(Y_test.shape)
print(X_test.shape)
# num_var = X_test.shape[1]


In [ ]:
#Import the regressors for the model
from sklearn.preprocessing import QuantileTransformer
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense
from keras.utils import np_utils
from keras import regularizers
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import metrics
import numpy as np

In [ ]:

# test_gen = TimeseriesGenerator(X_test,
#                                      X_test[:,0] ,
#                                      length=look_back,
#                                      batch_size=len(X_test),
#                                      reverse=False)
# # print(train_gen[0][0].shape)
# print(test_gen[0][0].shape)
# print(test_gen[0][0].shape[2])

train_gen = TimeseriesGenerator(X_train,
                                Y_train ,
                                length=look_back,
                                batch_size=int(len(X_train)/1),
                                reverse=False)

test_gen = TimeseriesGenerator(X_test,
                                     Y_test ,
                                     length=look_back,
                                     batch_size=len(X_test),
                                     reverse=False)
print(train_gen[0][0].shape)
print(test_gen[0][0].shape)

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
def loss_wrapper(beta):
    _beta = beta
    def customLoss(y_true,y_pred):
        if not K.is_tensor(y_pred):
            y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
#         print(y_true)
#         K.mean(K.square(y_pred - y_true), axis=-1)
        return K.mean(K.square(y_pred - y_true), axis=-1) + K.maximum(0.0,_beta*K.mean(-y_pred * y_true, axis=-1))

    return customLoss

In [ ]:
from keras.models import load_model
# custom_objects={'loss_max': loss_max}
# folder = '1monthfuturesforward-2_back-30_lmbda-1e-13_t_shape-uniform_f_shape-uniform_batch_size50'
folder = '1monthfutures_20100101_20170101_20180101_forward-2_back-30_lmbda-1e-09_t_shape-uniform_f_shape-uniform_lstm-1_batch_size50'
# folder = 'customLoss_1monthfutures_20100101_20170101_20180101_forward-2_back-30_lmbda-1e-10_t_shape-uniform_f_shape-uniform_lstm-2_batch_size100'
# folder = '1monthfuturesforward-15_back-15_lmbda-1e-09_t_shape-uniform_f_shape-uniform_batch_size10'
file = 'model-epoch073-loss0.0499.hdf5'
# file = 'model-epoch001-loss0.0503.hdf5'#Best
# file = 'model-epoch108-loss0.0498.hdf5'
# file = 'model-epoch182-loss0.0612.hdf5'
# file ='model-epoch083-loss0.0464.hdf5'
model = load_model("/Users/frizzell/forecast_models/{0}/{1}".format(folder,file),custom_objects={'customLoss': loss_wrapper(0.5)})

# model = load_model("/Users/frizzell/forecast_models/forward-5_back-5_lmbda-1e-09_t_shape-uniform_f_shape-uniform_batch_size50/model-epoch209-loss0.0523.hdf5")
# model = load_model("/Users/frizzell/forecast_models/all_variables_models/forward30_back30_model-034-0.5167.hdf5")

In [ ]:

np.random.seed(7)
# num_of_features = test_gen[0][0].shape[2]
# look_back=90
# window_size = look_back

# # expected input data shape: (batch_size, timesteps, data_dim)
# model = Sequential()
# model.add(LSTM(num_of_features*2,kernel_regularizer=regularizers.l1(0.0000001) ,
#                activity_regularizer=regularizers.l1(0.0000001),
#                return_sequences=True, input_shape=(window_size, num_of_features)))  # returns a sequence of vectors of dimension 32
# #model.add(SimpleRNN(num_of_features*2, return_sequences=True, input_shape=(window_size, num_of_features)))  # returns a sequence of vectors of dimension 32
# #model.add(LSTM(num_of_features*2, return_sequences=True))
# #model.add(LSTM(num_of_features , return_sequences=True))
# model.add(SimpleRNN(num_of_features,kernel_regularizer=regularizers.l2(0.0000001),
#                     activity_regularizer=regularizers.l1(0.0000001)))
# model.add(Dense(num_of_features,kernel_regularizer=regularizers.l2(0.0000001),
#                 activity_regularizer=regularizers.l1(0.0000001), activation='selu',))
# model.add(Dense(int(num_of_features/2),kernel_regularizer=regularizers.l2(0.0000001),
#                 activity_regularizer=regularizers.l1(0.0000001), activation='tanh'))
# model.add(Dense(2, activation='tanh',activity_regularizer=regularizers.l1(0.0000001)))
# model.add(Dense(1, activation='linear'))


# #sgd = SGD(lr=0.01, decay=1e-8, momentum=0.4, nesterov=True)
# #loss='mean_squared_error', optimizer='adam'
# model.compile(loss='mean_squared_error', optimizer='adam',
#               #optimizer=sgd,
#               metrics= [metrics.mse])

In [ ]:
#Build and train our models
# students = build_regressors(y_train.columns)
# smart_students, train_predictions = train_regressors(students,x_train,y_train)
# model_weights = "/Users/frizzell/forecast_models/back90_forward30_rnn.hdf"
# model.load_weights(model_weights)

In [ ]:
KPI = np.full((len(X_test),1),np.nan)
for x in range(look_back,len(X_test)):
    
    feature = X_test[x-look_back:x]
    KPI[x] =  model.predict( feature[np.newaxis,...] )

KPI_flat = KPI[:,0]

# KPI_mean = KPI_flat[~np.isnan(KPI_flat)].mean()
# KPI_std = KPI_flat[~np.isnan(KPI_flat)].std()

In [ ]:
indicators = target_transformer.inverse_transform( KPI_flat.reshape(-1,1))# / KPI_std
# plt.hist(indicators)
# plt.show()
# indicator

In [ ]:
indicators.shape

In [ ]:
#Dirty test to see what we are learning. 
#Essentially ifthe histogram is shaper, narrower, and more centered at zero it means we are more correct
#than if we simply guess the price in X days is the same as it is now
# ave = 4
# pred = pd.DataFrame(p).rolling(ave).mean()[ave:]
# plt.hist(t-p,bins=50)
# plt.show()
# print((t-p).mean())
# print((t-p).std())
# # print(t-p)
# for x in range(0,len(pred)):
#     if abs(pred[x]) > 0.5:
#         print(str(pred[x])+" " +str(t[x]-p[x]))

In [ ]:
#Lets have a peak at the regression predictions and make sure there are approximatly centered at zero
# for symbol in symbols:
#     short_plot(smart_students[symbol].predict(x_test) / train_predictions[symbol].std())


In [ ]:
# symbols

In [ ]:
backtest = df.iloc[-len(indicators):]
backtest["rnn_score"] = indicators


In [ ]:
#Build the dataframe panel to use for backtesting.
dataframes = []
panel_dictionary = {}
# pdb.set_trace()
for symbol in symbols:
    rnn_score = backtest["rnn_score"] 
#     df[symbol+" MovAveScore"] = signalMovAvrZscore.to_frame()
    temp = rnn_score.to_frame().dropna()
    temp.columns     = ["rnn_score"]
    temp["wheat_MA20"] = (backtest["wheat_settle"].rolling(2).mean() - backtest["wheat_settle"].mean() )/backtest["wheat_settle"].std()# Renormalize to gaussian using training std
#     df["VolumeScore"] = zScore( testSet[symbol]['volume'] ,10)
    temp["Symbol"]   = symbol
    temp["close"] = backtest[symbol]
    temp["date"]     = temp.index#.map(dateToStringF)
    dataframes.append(temp)
    panel_dictionary[symbol] = temp

    
do_spy = True
if do_spy:
    base = "/Users/frizzell/forecast_models/data"
    benchmark = pd.read_csv(base+"/benchmark_data_2020-01-10.csv")
    benchmark.index = pd.DatetimeIndex(benchmark.date)
    benchmark.index = pd.DatetimeIndex([datetime.datetime.strptime(str(x).split("T")[0],'%Y-%m-%d') for x in benchmark.index.values ])
    benchmark = benchmark.loc[temp.index]
    benchmark = benchmark[ ['adjClose','adjOpen'] ]
#     benchmark['change'] = (benchmark['adjClose']- benchmark['adjClose'].shift(1)).fillna(0)
    benchmark['change'] = benchmark['adjClose'].pct_change().fillna(0)#(benchmark['adjClose']- benchmark['adjClose'].shift(1)).fillna(0)
    benchmark["close"] = benchmark['adjClose']
    benchmark["open"] = benchmark["adjOpen"]
    benchmark["date"] = benchmark.index
    benchmark["Symbol"] = "benchmark"
    benchmark = benchmark.interpolate(method="time")
    panel_dictionary["benchmark"] = benchmark

result = pd.concat(dataframes)
backtestPanel = pd.Panel(panel_dictionary)
result.to_csv(signalsPath,index=False)

In [ ]:
# backtestPanel["benchmark"]
# print(benchmark.index)

In [ ]:
print(backtestPanel["wheat_settle"].rnn_score.std()) #old)

rnn_std = backtestPanel["wheat_settle"].rnn_score.std()

backtestPanel["wheat_settle"].head(3) #old

In [ ]:
# result['VolumeScore'].hist(bins=30)
# plt.show()
# result
# backtestPanel["wheat_settle"]
# backtestPanel["benchmark"]
# Asset('NYSE',)
# backtestPanel["benchmark"]

In [ ]:
from zipline.assets import Asset
SignalThreshold = rnn_std/2# sigma threshold for signal to open position
AveThreshold = 0.0

close_parameter = rnn_std/2
from six import viewkeys

# load signal to Zipline
def initialize(context):
    df = fetch_csv(signalsPath, symbol_column="Symbol")
    set_benchmark( zipline_symbol("benchmark") )
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    
#     print(context)
#     print(dir(context))
#     print(context.symbols)
    #df = pd.read_csv(signalsPath)#, symbol_column="Symbol")


    #print(strLocation()+str(df.df.head()))
    
# write simple signal-based logic - no hedge , longOnly
# open at z=-K, close at z=0 or z=-2*K 
def handle_correlations_aggressively(context, data):
    # check for all symbols signal value
    
    portfolio=context.portfolio
    record(positions=len(portfolio.positions.keys()))
    # print("Date = "+str(algo_obj.get_datetime()))
    
    for symb in ["wheat_settle",]: 

        sec = zipline_symbol(symb)
#         debug(sec)
        if (data[sec].rnn_score > SignalThreshold) and (data[sec].wheat_MA20 < -AveThreshold) and (sec not in viewkeys(portfolio.positions)):
#         if (data[sec].rnn_score > SignalThreshold) and (sec not in viewkeys(portfolio.positions)):


            #for asset in viewkeys(portfolio.positions):
            order_target_percent(sec, 0)
            order_target_percent(sec, 1.0) # long
            print("Lets buy "+symb+"  "+str(algo_obj.get_datetime()))
            
        if (data[sec].rnn_score < -SignalThreshold) and (data[sec].wheat_MA20 > AveThreshold) and (sec not in viewkeys(portfolio.positions)):
#         if (data[sec].rnn_score < -SignalThreshold) and (sec not in viewkeys(portfolio.positions)):

            #for asset in viewkeys(portfolio.positions):
            order_target_percent(sec, 0)
            order_target_percent(sec, -1.0) # full portfolio loaded with symb
            print("Lets short "+symb+"  "+str(algo_obj.get_datetime()))
            

        
    for asset in viewkeys(portfolio.positions):
        shares = portfolio.positions[asset].amount 
        #if (abs(data[asset].MVAscore) < close_parameter) : # close position logic - cross 0 or StopLoss
        if  ((shares > 0) and (data[asset].rnn_score < close_parameter)) \
        or  ((shares < 0) and (data[asset].rnn_score > -1.0*close_parameter)) : # close position logic - cross 0 or StopLoss
            order_target_percent(asset, 0)
            print("Lets close "+str(asset)+"  "+str(algo_obj.get_datetime()))  


def buy_hold(context, data):
    
    portfolio=context.portfolio
    record(positions=len(portfolio.positions.keys()))
    for symb in ["CXO"]:
        sec = zipline_symbol(symb)
        #order_target_percent(sec, 0 ) # full portfolio loaded with symb
        order_target_percent(sec, 1) # full portfolio loaded with symb




In [ ]:
# backtestPanel["benchmark"]['close'].tail(5)
# print( algo_obj.trading_calendar.schedule[str(start.date()):str(end.date())])

# str(start.date())
# backtestPanel["benchmark"][  ]
# algo_obj.trading_calendar.schedule[str(start.date()):str(end.date())].index
# benchmark_change.index
# backtestPanel["wheat_settle"].index
# backtestPanel["benchmark"]

In [ ]:
import pdb
start = datetime.datetime(2018, 2, 14, 0, 0, 0, 0, pytz.utc)
end = datetime.datetime(2020, 1, 10, 0, 0, 0, 0, pytz.utc)
# end = datetime.datetime(2019, 6, 30, 0, 0, 0, 0, pytz.utc)



sim_params = create_simulation_parameters(start=start,
                                          end=end,
                                          capital_base=float("100000"),
                                          data_frequency='daily',
                                          trading_calendar=None)

print("Starting simulation with ")
# print(sim_params)
algo_obj = TradingAlgorithm(initialize=initialize,
                            handle_data=handle_correlations_aggressively,
                            sim_params=sim_params)

benchmark_change = backtestPanel["benchmark"][[ str(x.date())  in algo_obj.trading_calendar.schedule[str(start.date()):str(end.date())].index for x in backtestPanel["benchmark"].index]]

# algo_obj.benchmark_hack = benchmark_change["change"]

perf_manual = algo_obj.run(backtestPanel)

In [ ]:
# print(perf_manual.benchmark_period_return.head(6))
# perf_manual.benchmark_period_return
perf_manual.benchmark_period_return.sum()

In [ ]:
# result['2017-01-03']
# history = pf.utils.extract_rets_pos_txn_from_zipline(pd.DataFrame(perf_manual))
# # history
# b = perf_manual.benchmark_period_return
# c = backtestPanel["benchmark"]["change"]
# for x in range(0,len(b)):
#     print( str(b[x])+"---"+str(c[x]) )
# perf_manual.beta

In [ ]:
# perf_manual.portfolio_value.plot()
perf_manual.returns.sum()
# perf_manual.portfolio_value[0]
# benchmark_offset
# spy = backtestPanel["benchmark"].close
# 100000 + backtestPanel["benchmark"].close.pct_change().map(lambda x:x*100000)


In [ ]:
#import matplotlib.pyplot as plt
import datetime
plt.clf()

fig, axs = plt.subplots(5)

fig.set_size_inches(16, 30, forward=True)

axs[0].plot( perf_manual.portfolio_value, label='Portfolio Value')
axs[0].set_ylabel('Portfolio Value', color='b')


axs02 = axs[0].twinx() 
axs02.plot( backtestPanel.benchmark.close , label='SPY+',color='r')
axs02.set_ylim(233,649)
axs02.set_ylabel('SPY', color='r')
axs[0].legend()
axs02.legend(loc='lower right')
axs[0].set_xlim([datetime.date(2018, 3, 1),datetime.date(2020, 1, 21)])
axs[0].set_ylim([90000,250000])
# plt.show()
# plt.subplot(1,5,2)


# axs[1].figure().set_size_inches(16, 6, forward=True)
axs[1].plot( backtestPanel["wheat_settle"].close , label='Market Price' )

axs[1].legend()
axs[1].set_xlim([datetime.date(2018, 3, 1),datetime.date(2020, 1, 21)])
# plt.show()
# plt.subplot(1,5,3)


# axs[2].figure().set_size_inches(16, 6, forward=True)
axs[2].plot( backtestPanel["wheat_settle"].rnn_score, label='RNN Score')
axs[2].plot( backtestPanel["wheat_settle"].wheat_MA20, label='MA20')
axs[2].legend()
axs[2].set_xlim([datetime.date(2018, 3, 1),datetime.date(2020, 1, 21)])
# plt.show()
# plt.subplot(1,5,4)



# axs[3].figure().set_size_inches(16, 6, forward=True)
axs[3].plot( perf_manual.ending_value, label='Position Value')
axs[3].legend()
axs[3].set_xlim([datetime.date(2018, 3, 1),datetime.date(2020, 1, 21)])
axs[3].set_ylim([-270000,270000])
# plt.show()
# plt.subplot(1,5,5)

# axs[4].figure().set_size_inches(16, 6, forward=True)
axs[4].plot( perf_manual.positions, label='# of Positions')
axs[4].legend()
axs[4].set_xlim([datetime.date(2018, 3, 1),datetime.date(2020, 1, 21)])

plt.show()



In [ ]:
# perf_manual.alpha.plot()
print(perf_manual.alpha.mean())
plt.plot(perf_manual.alpha.rolling(100).mean().dropna())
plt.show()
perf_manual.beta.plot()
print(perf_manual.beta.mean())
plt.plot(perf_manual.beta.rolling(100).mean().dropna())
plt.show()
# pf.create_simple_tear_sheet(perf_manual.returns)
# returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf_manual.returns)

In [ ]:
pf.create_full_tear_sheet(perf_manual.returns)

In [ ]:
trades = [ x for x in perf_manual.transactions if x != [] ]
# trades

In [ ]:
trades_summary = [] 
for x in range(0,len(trades),2):
    if x == len(trades)-1:
        break
    enter = trades[x][0]
    exit = trades[x+1][0]
    trades_summary.append(-1.0*(enter["amount"] * enter["price"] + exit["amount"] * exit["price"]))
    
# trades_summary = perf_manual.returns.values
print( sum(trades_summary) )

In [ ]:
# trades_summary

In [ ]:
trades_summary = np.asarray(trades_summary)
plt.hist(trades_summary,bins=30)
plt.show()

In [ ]:
num_throws = 1000000

bootstraps = np.full((num_throws,len(trades_summary)),np.nan)

for x in range(0,num_throws):
    
    bootstraps[x] = np.random.choice(trades_summary,len(trades_summary))
        
    

In [ ]:
# bootstraps

In [ ]:
plt.hist(bootstraps[:,-1],bins=40)
plt.show()

In [ ]:
def max_drawdown(array):
    accumulator = 0 
    dips = [] 
    for x in array:
        if x < 0:
            accumulator = accumulator + x
        else:
            dips.append( abs(accumulator) )
            accumulator = 0
#     print(dips)
    return max(dips)

draw_downs = []

for sampling in bootstraps:
    draw_downs.append( max_drawdown(sampling) )

In [ ]:
plt.hist(draw_downs,bins=30)
plt.show()
# draw_downs

In [ ]:
def make_MC(transactions):
    init = 100000
    simulation = [init,]

    for x in transactions:
        simulation.append(init+x)
        init = init+x
    return simulation

for i,strap in enumerate(bootstraps):
    if not i % 4000:
        plt.plot(make_MC(strap))
    
    
plt.plot(make_MC(trades_summary), color='black', linewidth=3)
plt.ylabel("Value")
plt.xlabel("# Trades")
plt.title("Monte Carlo Simulation")
plt.show()

In [ ]:
[d for d in draw_downs if d != 0 ]
len(draw_downs)
short = [d for d in draw_downs if not d % 10 ]
len(short)
# draw_downs

In [ ]:
import scipy
plt.clf()
print(scipy.stats.lognorm._fitstart(draw_downs))
muX = np.mean(np.log([d+1 for d in draw_downs]))
# sigmaX = np.std(np.log(draw_downs))
print(muX)
shape,loc,scale = scipy.stats.lognorm.fit([d for d in draw_downs if d != 0 ],floc=0,scale=6000)#,scale=muX)
linespace = np.linspace(0,40000,10000)

print( np.asarray(draw_downs).mean())
print( np.asarray(draw_downs).std())
print(shape,loc,scale)


pdf = scipy.stats.lognorm.pdf(linespace, shape, loc, scale)

# dist = scipy.stats.lognorm([stddev],loc=mean)

plt.hist( draw_downs, bins = 10 ,normed=True)
plt.plot(linespace, pdf )
# plt.plot(linespace,lognorm.pdf(linespace))
# shape, floc, scale
plt.show()
cdf = scipy.stats.lognorm.cdf(linespace, shape, loc, scale)
plt.plot( linespace, cdf ) 

conf = 0
for x,num in zip(linespace,cdf):
    if num > .95:
        print("p val = 0.95 at : " + str(x))
        conf = x
        break
        
# plt.plot(conf,[1,])
plt.axvline(x=conf, color='k', linestyle='--')
plt.show()



